### **ENVIRONMENT SETUP**

In [3]:
# ! rm -r data*
# ! wget http://argumentation.bplaced.net/arguana-data/dagstuhl-15512-argquality-corpus-v2.zip
# ! unzip dagstuhl-15512-argquality-corpus-v2.zip
# ! rm *.zip
# ! rm -r __MACOSX
# ! mv dagstuhl-15512-argquality-corpus-v2 data

### **IMPORT LIBRARIES**

In [4]:
# Set random seed

import random
random.seed(14071)

In [33]:
import re
import pandas as pd
import numpy as np
import pickle

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer # Bag of Words
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
from sklearn.model_selection import cross_val_predict # Cross Validation
from sklearn.model_selection import KFold # K Cross

from sklearn.preprocessing import LabelEncoder # Label Encoding
from sklearn.preprocessing import OneHotEncoder # One Hot Encoding

from sklearn.model_selection import train_test_split # Train Test Split

from sklearn.linear_model import LogisticRegression # LR Model
from sklearn.tree import DecisionTreeClassifier # DT Model
from sklearn.ensemble import RandomForestClassifier # RF Model

import tensorflow as tf # Tensorflow bindings
from tensorflow import keras # Keras bindings

from sklearn.metrics import classification_report # Classification Report

from sklearn.model_selection import GridSearchCV # Grid Search

[nltk_data] Downloading package stopwords to /home/sri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sri/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### **IMPORT DATA**

In [7]:
df = pd.read_csv("data/dagstuhl-15512-argquality-corpus-annotated.csv", sep='\t', encoding_errors="ignore")

In [8]:
df

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance
0,1,y,1 (Low),1 (Low),1 (Low),1 (Low),2 (Average),1 (Low),1 (Low),1 (Low),...,1 (Low),1 (Low),1 (Low),1 (Low),1 (Low),1 (Low),"it is true that bottled water is a waste, but ...",arg219250,ban-plastic-water-bottles,no-bad-for-the-economy
1,2,y,1 (Low),3 (High),2 (Average),2 (Average),3 (High),1 (Low),1 (Low),3 (High),...,1 (Low),2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),"it is true that bottled water is a waste, but ...",arg219250,ban-plastic-water-bottles,no-bad-for-the-economy
2,3,y,2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),2 (Average),1 (Low),2 (Average),"it is true that bottled water is a waste, but ...",arg219250,ban-plastic-water-bottles,no-bad-for-the-economy
3,1,y,2 (Average),3 (High),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),3 (High),2 (Average),2 (Average),Most Americans on average recycle 86-88% of th...,arg219293,ban-plastic-water-bottles,no-bad-for-the-economy
4,2,y,1 (Low),2 (Average),1 (Low),2 (Average),2 (Average),1 (Low),1 (Low),2 (Average),...,1 (Low),1 (Low),2 (Average),2 (Average),2 (Average),1 (Low),Most Americans on average recycle 86-88% of th...,arg219293,ban-plastic-water-bottles,no-bad-for-the-economy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2,y,2 (Average),2 (Average),3 (High),2 (Average),2 (Average),1 (Low),1 (Low),3 (High),...,1 (Low),2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),Raffles neglected Singapore when he went aroun...,arg168822,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course
956,3,y,2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),2 (Average),2 (Average),2 (Average),Raffles neglected Singapore when he went aroun...,arg168822,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course
957,1,y,2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),2 (Average),2 (Average),2 (Average),...,2 (Average),2 (Average),3 (High),2 (Average),2 (Average),2 (Average),"Raffles doesn't care about the citizens, doesn...",arg168834,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course
958,2,y,2 (Average),2 (Average),3 (High),2 (Average),3 (High),1 (Low),2 (Average),2 (Average),...,1 (Low),2 (Average),2 (Average),2 (Average),3 (High),1 (Low),"Raffles doesn't care about the citizens, doesn...",arg168834,william-farquhar-ought-to-be-honoured-as-the-r...,yes-of-course


In [9]:
print(f"Number of annotations = {len(df['argument'])}")
print(f"Number of unique arguements = {len(np.unique(df['argument']))}") # Each argument was scored by 3 annotators
print(f"Number of unique issue = {len(np.unique(df['issue']))}")  # There are a total of 16 issues
print(f"Number of unique stance = {len(np.unique(df['stance']))}") # Each issue has on an avg 2 stance (positive and negative)

Number of annotations = 960
Number of unique arguements = 320
Number of unique issue = 16
Number of unique stance = 28


### **DATA CLEANING**

#### Remove statements are that are tagged as NOT argumentative

In [10]:
df[df["argumentative"] == "n"] # Statements that are tagged as NOT argumentative

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance
25,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,We will be able to ban water bottles until we ...,arg219242,ban-plastic-water-bottles,no-bad-for-the-economy
32,3,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The high price of bottled water is not the wat...,arg219232,ban-plastic-water-bottles,yes-emergencies-only
37,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,A drop of water is worth more than a sack of g...,arg219210,ban-plastic-water-bottles,yes-emergencies-only
51,1,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yeah I have a bottle of water next to me its n...,arg219292,ban-plastic-water-bottles,yes-emergencies-only
52,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yeah I have a bottle of water next to me its n...,arg219292,ban-plastic-water-bottles,yes-emergencies-only
53,3,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yeah I have a bottle of water next to me its n...,arg219292,ban-plastic-water-bottles,yes-emergencies-only
97,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,This is just wrong we should not insult who we...,arg236317,christianity-or-atheism,christianity
104,3,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I have a personal relationship with Christ. I ...,arg317490,christianity-or-atheism,christianity
105,1,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,God helps those who help themselves! So i will...,arg234318,christianity-or-atheism,christianity
106,2,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,God helps those who help themselves! So i will...,arg234318,christianity-or-atheism,christianity


In [11]:
statements = df[df["argumentative"] == "n"]["argument"].to_numpy() # Extract the statements
statements = np.unique(statements) # Extract the unique statements

for ele in statements: # Remove all occurrences of NOT argumentative statements
    df.drop(df[df['argument'] == ele].index, axis=0, inplace=True)

In [12]:
df[df["argumentative"] == "n"]

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance


In [13]:
print(f"Number of annotations = {len(df['argument'])}")
print(f"Number of unique arguements = {len(np.unique(df['argument']))}") # Each argument was scored by 3 annotators
print(f"Number of unique issue = {len(np.unique(df['issue']))}")  # There are a total of 16 issues
print(f"Number of unique stance = {len(np.unique(df['stance']))}") # Each issue has on an avg 2 stance (positive and negative)

Number of annotations = 912
Number of unique arguements = 304
Number of unique issue = 16
Number of unique stance = 28


#### Combine all Annotators' scores into a single score

In [14]:
argument = np.unique(df["argument"])

In [15]:
attributes = ["annotator", "overall quality", "cogency", "effectiveness", "reasonableness", "argument", "#id"]

cleaned_df = []

for arg in argument:

    new_df = df[df["argument"] == arg][attributes]
    flag = 0
    new_dict = {
        "#id": new_df["#id"].iloc[0],
        "argument": new_df["argument"].iloc[0],
    }

    for ele in ["overall quality", "cogency", "effectiveness", "reasonableness"]:
        if len(pd.value_counts(new_df[ele])) == 3:
            flag = 1
            break
        new_dict[ele] = pd.value_counts(new_df[ele]).index[0]
        
    if flag == 1:
        continue
    cleaned_df.append(new_dict)

cleaned_df = pd.DataFrame(cleaned_df)

In [16]:
df[df["#id"] == "28068"]

,annotator,argumentative,overall quality,local acceptability,appropriateness,arrangement,clarity,cogency,effectiveness,global acceptability,...,global sufficiency,reasonableness,local relevance,credibility,emotional appeal,sufficiency,argument,#id,issue,stance
144,1,y,2 (Average),2 (Average),2 (Average),2 (Average),2 (Average),1 (Low),1 (Low),1 (Low),...,1 (Low),1 (Low),3 (High),1 (Low),2 (Average),1 (Low),"""Debates are based on convincing evidence. The...",28068,evolution-vs-creation,creation
145,2,y,1 (Low),1 (Low),3 (High),2 (Average),2 (Average),1 (Low),1 (Low),1 (Low),...,1 (Low),1 (Low),2 (Average),1 (Low),2 (Average),1 (Low),"""Debates are based on convincing evidence. The...",28068,evolution-vs-creation,creation
146,3,y,2 (Average),2 (Average),2 (Average),2 (Average),3 (High),2 (Average),1 (Low),2 (Average),...,1 (Low),1 (Low),2 (Average),1 (Low),2 (Average),2 (Average),"""Debates are based on convincing evidence. The...",28068,evolution-vs-creation,creation


In [17]:
cleaned_df

,#id,argument,overall quality,cogency,effectiveness,reasonableness
0,28068,"""Debates are based on convincing evidence. The...",2 (Average),1 (Low),1 (Low),1 (Low)
1,13270,"""If a women is raped"" is a good argument. Howe...",1 (Low),1 (Low),1 (Low),1 (Low)
2,13275,"""The government has no place to tell a woman w...",1 (Low),1 (Low),1 (Low),1 (Low)
3,12365,(I am writing this through Firefox) Emotions a...,2 (Average),1 (Low),1 (Low),2 (Average)
4,arg561672,1. It makes everyone equal - if children can w...,1 (Low),2 (Average),1 (Low),2 (Average)
...,...,...,...,...,...,...
256,arg334959,"yea, because even though there are many other ...",2 (Average),2 (Average),1 (Low),2 (Average)
257,arg335089,yes because if they fear getting hit than they...,1 (Low),1 (Low),1 (Low),1 (Low)
258,arg203922,"yes, i believe it's nice to have a school unif...",2 (Average),2 (Average),1 (Low),2 (Average)
259,arg596217,"yes,India has potential to lead the world.So, ...",1 (Low),1 (Low),1 (Low),1 (Low)


In [18]:
print(f"Number of arguements = {len(cleaned_df['argument'])}")

Number of arguements = 261


### **DATA PREPROCESSING**

In [19]:
text = cleaned_df["argument"]

In [20]:
stop_words = set(stopwords.words("english"))
english_stopwords = stopwords.words("english")
english_stemmer = SnowballStemmer("english")

In [21]:
def clean_text(text):
    text = text.replace('</br>', '') # Remove </br>
    text = re.sub(r'[^\w]', ' ', text) # Remove symbols
    text = re.sub(r'[ ]{2,}', ' ', text) # Remove extra spaces
    text = re.sub(r'[ \t]+$', '', text) # Remove trailing white spaces
    tokens = []
    for token in text.split():
        if token not in stop_words:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)
    #return token

In [22]:
cleaned_text = [clean_text(text) for text in text]
text = cleaned_text

### **VECTORIZE THE TEXT DATA**

In [23]:
# Using Bag of Words (BoW)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

print(f"Shape of Vector = {X.shape}")

Shape of Vector = (261, 2001)


### **PREDICTING COGENCY FROM TEXT**

In [29]:
y = cleaned_df["cogency"].to_numpy()
# y = y.reshape(-1, 1)

#### Label Encoding

In [30]:
# Label Encoding

encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y

print(f"Size of Labels = {y.shape}")

Size of Labels = (261,)


In [31]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2001)
Shape of Training Labels: (261,)


#### Training Logistic Regression Model

In [34]:
model = LogisticRegression(dual=False, fit_intercept=True, penalty="none", solver="sag", max_iter=5000)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.63      0.77      0.69       147
           1       0.49      0.40      0.44        97
           2       0.00      0.00      0.00        17

    accuracy                           0.58       261
   macro avg       0.37      0.39      0.38       261
weighted avg       0.53      0.58      0.55       261



#### Training Decision Tree Model

In [37]:
model = DecisionTreeClassifier(criterion="gini", max_features="log2", splitter="best")
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.58      0.71      0.64       147
           1       0.38      0.28      0.32        97
           2       0.25      0.12      0.16        17

    accuracy                           0.51       261
   macro avg       0.40      0.37      0.37       261
weighted avg       0.48      0.51      0.49       261



#### Training Random Forest Model

In [36]:
model = RandomForestClassifier(bootstrap=True, class_weight='balanced', 
    criterion='gini', max_features=None, n_estimators=300, oob_score=False, warm_start=False)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.61      0.78      0.69       147
           1       0.44      0.33      0.38        97
           2       0.00      0.00      0.00        17

    accuracy                           0.56       261
   macro avg       0.35      0.37      0.36       261
weighted avg       0.51      0.56      0.53       261



#### Training Neural Network

In [38]:
# One Hot Encoding

y = cleaned_df["cogency"].to_numpy()
y = y.reshape(-1, 1)

encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y.toarray()

print(f"Size of Labels = {y.shape}")
print(f"Label Sample: {y[0]}")

Size of Labels = (261, 3)
Label Sample: [1. 0. 0.]


In [39]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2001)
Shape of Training Labels: (261, 3)


In [40]:
def init_model():

    # Define Model
    model = keras.models.Sequential([
        keras.layers.Dense(32, input_dim=X.shape[1], activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation="softmax"),
    ])

    loss_function = keras.losses.CategoricalCrossentropy() # Define loss function
    # loss_function = keras.losses.SparseCategoricalCrossentropy() # Define loss function
    optimizer = keras.optimizers.SGD(learning_rate=0.005) # Define optimizer

    model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"]) # Compile the model

    return model

In [42]:
# Train the model

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = init_model()

    history = model.fit(X_train, y_train, epochs=75, batch_size=1, 
        validation_data=(X_test, y_test), verbose=0)

    pred_test = [np.argmax(ele) for ele in model.predict(X_test)]
    pred += pred_test

2/2 [==============================] - 0s 3ms/step


In [43]:
new_y = [np.argmax(ele) for ele in y]
print(classification_report(new_y, pred))

              precision    recall  f1-score   support

           0       0.65      0.71      0.68       147
           1       0.48      0.48      0.48        97
           2       0.00      0.00      0.00        17

    accuracy                           0.58       261
   macro avg       0.38      0.40      0.39       261
weighted avg       0.55      0.58      0.56       261



### **PREDICTING EFFECTIVENESS FROM TEXT**

In [45]:
y = cleaned_df["effectiveness"].to_numpy()
# y = y.reshape(-1, 1)

#### Label Encoding

In [46]:
# Label Encoding

encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y

print(f"Size of Labels = {y.shape}")

Size of Labels = (261,)


In [47]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2001)
Shape of Training Labels: (261,)


#### Training Logistic Regression Model

In [48]:
model = LogisticRegression(dual=False, fit_intercept=True, penalty="none", solver="sag", max_iter=5000)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.67      0.80      0.73       172
           1       0.33      0.23      0.27        80
           2       0.00      0.00      0.00         9

    accuracy                           0.60       261
   macro avg       0.33      0.34      0.33       261
weighted avg       0.54      0.60      0.56       261



/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr

#### Training Decision Tree Model

In [49]:
model = DecisionTreeClassifier(criterion="gini", max_features="log2", splitter="best")
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.66      0.77      0.71       172
           1       0.33      0.23      0.27        80
           2       0.00      0.00      0.00         9

    accuracy                           0.58       261
   macro avg       0.33      0.33      0.33       261
weighted avg       0.54      0.58      0.55       261



#### Training Random Forest Model

In [50]:
model = RandomForestClassifier(bootstrap=True, class_weight='balanced', 
    criterion='gini', max_features=None, n_estimators=300, oob_score=False, warm_start=False)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.68      0.88      0.77       172
           1       0.43      0.20      0.27        80
           2       0.00      0.00      0.00         9

    accuracy                           0.64       261
   macro avg       0.37      0.36      0.35       261
weighted avg       0.58      0.64      0.59       261



#### Training Neural Network

In [51]:
# One Hot Encoding

y = cleaned_df["effectiveness"].to_numpy()
y = y.reshape(-1, 1)

encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y.toarray()

print(f"Size of Labels = {y.shape}")
print(f"Label Sample: {y[0]}")

Size of Labels = (261, 3)
Label Sample: [1. 0. 0.]


In [52]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2001)
Shape of Training Labels: (261, 3)


In [53]:
def init_model():

    # Define Model
    model = keras.models.Sequential([
        keras.layers.Dense(32, input_dim=X.shape[1], activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation="softmax"),
    ])

    loss_function = keras.losses.CategoricalCrossentropy() # Define loss function
    # loss_function = keras.losses.SparseCategoricalCrossentropy() # Define loss function
    optimizer = keras.optimizers.SGD(learning_rate=0.005) # Define optimizer

    model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"]) # Compile the model

    return model

In [54]:
# Train the model

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = init_model()

    history = model.fit(X_train, y_train, epochs=75, batch_size=1, 
        validation_data=(X_test, y_test), verbose=0)

    pred_test = [np.argmax(ele) for ele in model.predict(X_test)]
    pred += pred_test

2/2 [==============================] - 0s 4ms/step


In [55]:
new_y = [np.argmax(ele) for ele in y]
print(classification_report(new_y, pred))

              precision    recall  f1-score   support

           0       0.67      0.78      0.72       172
           1       0.34      0.25      0.29        80
           2       0.00      0.00      0.00         9

    accuracy                           0.59       261
   macro avg       0.34      0.34      0.34       261
weighted avg       0.54      0.59      0.56       261



### **PREDICTING REASONABLENESS FROM TEXT**

In [56]:
y = cleaned_df["reasonableness"].to_numpy()
# y = y.reshape(-1, 1)

#### Label Encoding

In [57]:
# Label Encoding

encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y

print(f"Size of Labels = {y.shape}")

Size of Labels = (261,)


#### Train Test Split

In [58]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2001)
Shape of Training Labels: (261,)


#### Training Logistic Regression Model

In [59]:
model = LogisticRegression(dual=False, fit_intercept=True, penalty="none", solver="sag", max_iter=5000)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.54      0.64      0.59       124
           1       0.54      0.50      0.51       123
           2       0.00      0.00      0.00        14

    accuracy                           0.54       261
   macro avg       0.36      0.38      0.37       261
weighted avg       0.51      0.54      0.52       261



#### Training Decision Tree Model

In [60]:
model = DecisionTreeClassifier(criterion="gini", max_features="log2", splitter="best")
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.53      0.67      0.59       124
           1       0.58      0.46      0.51       123
           2       0.00      0.00      0.00        14

    accuracy                           0.53       261
   macro avg       0.37      0.37      0.37       261
weighted avg       0.53      0.53      0.52       261



#### Training Random Forest Model

In [61]:
model = RandomForestClassifier(bootstrap=True, class_weight='balanced', 
    criterion='gini', max_features=None, n_estimators=300, oob_score=False, warm_start=False)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.53      0.67      0.59       124
           1       0.52      0.43      0.47       123
           2       0.00      0.00      0.00        14

    accuracy                           0.52       261
   macro avg       0.35      0.37      0.35       261
weighted avg       0.50      0.52      0.50       261



#### Training Neural Network

In [66]:
# One Hot Encoding

y = cleaned_df["reasonableness"].to_numpy()
y = y.reshape(-1, 1)

encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y.toarray()

print(f"Size of Labels = {y.shape}")
print(f"Label Sample: {y[0]}")

Size of Labels = (261, 3)
Label Sample: [1. 0. 0.]


In [67]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2001)
Shape of Training Labels: (261, 3)


In [68]:
def init_model():

    # Define Model
    model = keras.models.Sequential([
        keras.layers.Dense(32, input_dim=X.shape[1], activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation="softmax"),
    ])

    loss_function = keras.losses.CategoricalCrossentropy() # Define loss function
    # loss_function = keras.losses.SparseCategoricalCrossentropy() # Define loss function
    optimizer = keras.optimizers.SGD(learning_rate=0.005) # Define optimizer

    model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"]) # Compile the model

    return model

In [69]:
# Train the model

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = init_model()

    history = model.fit(X_train, y_train, epochs=75, batch_size=1, 
        validation_data=(X_test, y_test), verbose=0)

    pred_test = [np.argmax(ele) for ele in model.predict(X_test)]
    pred += pred_test

2/2 [==============================] - 0s 3ms/step


In [71]:
new_y = [np.argmax(ele) for ele in y]
print(classification_report(new_y, pred))

              precision    recall  f1-score   support

           0       0.58      0.63      0.60       124
           1       0.58      0.59      0.59       123
           2       0.00      0.00      0.00        14

    accuracy                           0.58       261
   macro avg       0.39      0.41      0.40       261
weighted avg       0.55      0.58      0.56       261



/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr

### **PREDICTING OVERALL QUALITY FROM TEXT + ATTRIBUTES**

In [72]:
y = cleaned_df["overall quality"].to_numpy()
# y = y.reshape(-1, 1)

#### Adding the Attributes to Argument Vector

In [73]:
# Vectorize the Arguments

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

print(f"Initial shape of Vector = {X.shape}")

Initial shape of Vector = (261, 2001)


In [74]:
attr_enc_map = {
    "1 (Low)": np.array([0, 0, 1]),
    "2 (Average)": np.array([0, 1, 0]),
    "3 (High)": np.array([1, 0, 0]),
}

In [75]:
def encode(array):
    temp_list = []
    for ele in array:
        temp_list.append(attr_enc_map[ele])
    return np.array(temp_list)

In [76]:
cogency = cleaned_df["cogency"].to_numpy()
effectiveness = cleaned_df["effectiveness"].to_numpy()
reasonableness = cleaned_df["reasonableness"].to_numpy()

cogency = encode(cogency)
effectiveness = encode(effectiveness)
reasonableness = encode(reasonableness)

X_new = []

for idx, x in enumerate(X):
    temp = np.concatenate((cogency[idx], effectiveness[idx], reasonableness[idx], x))
    X_new.append(temp)

X = np.array(X_new)

print(f"Final shape of Vector = {X.shape}")

Final shape of Vector = (261, 2010)


#### Label Encoding

In [77]:
# Label Encoding

encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y

print(f"Size of Labels = {y.shape}")

Size of Labels = (261,)


#### Train Test Split

In [78]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2010)
Shape of Training Labels: (261,)


#### Training Logistic Regression Model

In [86]:
model = LogisticRegression(C=0.1, dual=False, fit_intercept=True, penalty="l2", solver="newton-cg")
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92       149
           1       0.81      0.85      0.83        96
           2       1.00      0.19      0.32        16

    accuracy                           0.86       261
   macro avg       0.90      0.66      0.69       261
weighted avg       0.87      0.86      0.85       261



#### Training Decision Tree Model

In [84]:
model = DecisionTreeClassifier(criterion="entropy", max_features="sqrt", splitter="best")
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86       149
           1       0.81      0.70      0.75        96
           2       0.79      0.69      0.73        16

    accuracy                           0.82       261
   macro avg       0.81      0.76      0.78       261
weighted avg       0.82      0.82      0.81       261



#### Training Random Forest Model

In [88]:
model = RandomForestClassifier(bootstrap=True, class_weight='balanced', 
    criterion='gini', max_features=None, n_estimators=300, oob_score=False, warm_start=False)
pred = cross_val_predict(model, X, y, cv=5)
pred = cross_val_predict(model, X, y, cv=5)
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90       149
           1       0.80      0.84      0.82        96
           2       0.78      0.88      0.82        16

    accuracy                           0.87       261
   macro avg       0.83      0.87      0.85       261
weighted avg       0.87      0.87      0.87       261



#### Training Neural Network

In [89]:
# One Hot Encoding

y = cleaned_df["overall quality"].to_numpy()
y = y.reshape(-1, 1)

encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
y = enc_y.toarray()

print(f"Size of Labels = {y.shape}")
print(f"Label Sample: {y[0]}")

Size of Labels = (261, 3)
Label Sample: [0. 1. 0.]


In [90]:
print(f"Shape of Training Data: {X.shape}")
print(f"Shape of Training Labels: {y.shape}")

Shape of Training Data: (261, 2010)
Shape of Training Labels: (261, 3)


In [91]:
def init_model():

    # Define Model
    model = keras.models.Sequential([
        keras.layers.Dense(32, input_dim=X.shape[1], activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.6),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation="softmax"),
    ])

    loss_function = keras.losses.CategoricalCrossentropy() # Define loss function
    # loss_function = keras.losses.SparseCategoricalCrossentropy() # Define loss function
    optimizer = keras.optimizers.SGD(learning_rate=0.005) # Define optimizer

    model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"]) # Compile the model

    return model

In [92]:
# Train the model

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = init_model()

    history = model.fit(X_train, y_train, epochs=75, batch_size=1, 
        validation_data=(X_test, y_test), verbose=0)

    pred_test = [np.argmax(ele) for ele in model.predict(X_test)]
    pred += pred_test

2/2 [==============================] - 0s 4ms/step


In [93]:
new_y = [np.argmax(ele) for ele in y]
print(classification_report(new_y, pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92       149
           1       0.83      0.89      0.85        96
           2       0.83      0.31      0.45        16

    accuracy                           0.87       261
   macro avg       0.86      0.71      0.74       261
weighted avg       0.87      0.87      0.87       261



### **BUILDING THE 2-LAYER MODEL**

#### Neural Network to Predict the Cogency

In [ ]:
# Vectorize the Arguments
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

# Extract Cogency
y = cleaned_df["cogency"].to_numpy()
y = y.reshape(-1, 1)

# Label Encoding
encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
enc_y = enc_y.toarray()

# Train Test Split
X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

# Define Custom Callback
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold
    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

# Define Model
cog_model = keras.models.Sequential([
    keras.layers.Dense(32, input_dim=X_train.shape[1], activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation="softmax"),
])

# Define Parameters
loss_function = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.SGD(learning_rate=0.005)
callback = MyThresholdCallback(threshold=0.66)

# Compile Model
cog_model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"])

# Train the model
history = cog_model.fit(X_train, y_train, epochs=1000, batch_size=1, 
    validation_data=(X_test, y_test), callbacks=[callback], verbose=0)

# Print Validation Accurace of Model
print(f"Validation Accuracy of Model = {cog_model.evaluate(X_test, y_test, verbose=0)[1]}")

#### Neural Network to Predict the Effectiveness

In [ ]:
# Vectorize the Arguments
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

# Extract Effectiveness
y = cleaned_df["effectiveness"].to_numpy()
y = y.reshape(-1, 1)

# Label Encoding
encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
enc_y = enc_y.toarray()

# Train Test Split
X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

# Define Custom Callback
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold
    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

# Define Model
eff_model = keras.models.Sequential([
    keras.layers.Dense(32, input_dim=X_train.shape[1], activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation="softmax"),
])

# Define Parameters
loss_function = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.SGD(learning_rate=0.005)
callback = MyThresholdCallback(threshold=0.66)

# Compile Model
eff_model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"])

# Train the model
history = eff_model.fit(X_train, y_train, epochs=1000, batch_size=1, 
    validation_data=(X_test, y_test), callbacks=[callback], verbose=0)

# Print Validation Accurace of Model
print(f"Validation Accuracy of Model = {eff_model.evaluate(X_test, y_test, verbose=0)[1]}")

#### Neural Network to Predict the Reasonableness

In [ ]:
# Vectorize the Arguments
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

# Extract Reasonableness
y = cleaned_df["reasonableness"].to_numpy()
y = y.reshape(-1, 1)

# Label Encoding
encoder = OneHotEncoder()
enc_y = encoder.fit_transform(y)
enc_y = enc_y.toarray()

# Train Test Split
X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

# Define Custom Callback
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold
    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

# Define Model
reas_model = keras.models.Sequential([
    keras.layers.Dense(32, input_dim=X_train.shape[1], activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation="softmax"),
])

# Define Parameters
loss_function = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.SGD(learning_rate=0.005)
callback = MyThresholdCallback(threshold=0.66)

# Compile Model
reas_model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"])

# Train the model
history = reas_model.fit(X_train, y_train, epochs=1000, batch_size=1, 
    validation_data=(X_test, y_test), callbacks=[callback], verbose=0)

# Print Validation Accurace of Model
print(f"Validation Accuracy of Model = {reas_model.evaluate(X_test, y_test, verbose=0)[1]}")

#### Logistic Regression Model to Predict the Overall Quality

In [ ]:
# Vectorize the Arguments
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

# Extracting the attributes
cogency = cleaned_df["cogency"].to_numpy()
effectiveness = cleaned_df["effectiveness"].to_numpy()
reasonableness = cleaned_df["reasonableness"].to_numpy()

attr_enc_map = {
    "1 (Low)": np.array([0, 0, 1]),
    "2 (Average)": np.array([0, 1, 0]),
    "3 (High)": np.array([1, 0, 0]),
}

def encode(array):
    temp_list = []
    for ele in array:
        temp_list.append(attr_enc_map[ele])
    return np.array(temp_list)

cogency = encode(cogency)
effectiveness = encode(effectiveness)
reasonableness = encode(reasonableness)

# Adding the attributes to argument vector
X_new = []
for idx, x in enumerate(X):
    temp = np.concatenate((cogency[idx], effectiveness[idx], reasonableness[idx], x))
    X_new.append(temp)
X = np.array(X_new)

# Extract Overall Quality
y = cleaned_df["overall quality"].to_numpy()
y = y.reshape(-1, )

# Label Encoding
encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)

# Train Test Split
X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

# Train Logistic Regression Model
qual_model = LogisticRegression(C=0.1, dual=False, fit_intercept=True,
    penalty="l2", solver="newton-cg")
qual_model.fit(X=X_train, y=y_train)

# Print Validation Accuracy of Model
pred_test = qual_model.predict(X_test)
print(f"Validation Accuracy of Model = {classification_report(y_test, pred_test, output_dict=True)['accuracy']}")

### **SAVE MODELS**

In [ ]:
pickle.dump(cog_model, open("models/cog_model.pkl", "wb"))
pickle.dump(eff_model, open("models/eff_model.pkl", "wb"))
pickle.dump(reas_model, open("models/reas_model.pkl", "wb"))
pickle.dump(qual_model, open("models/qual_model.pkl", "wb"))

### **DEFINE 2-LAYER MODEL**

In [47]:
class CustomModel:

    def __init__(self):
        self.attr_1_model = pickle.load(open("models/cog_model.pkl", "rb"))
        self.attr_2_model = pickle.load(open("models/eff_model.pkl", "rb"))
        self.attr_3_model = pickle.load(open("models/reas_model.pkl", "rb"))
        self.arg_model = pickle.load(open("models/qual_model.pkl", "rb"))

    def predict(self, array):
        attr_1 = self.attr_1_model.predict(array, verbose=0)
        attr_2 = self.attr_2_model.predict(array, verbose=0)
        attr_3 = self.attr_3_model.predict(array, verbose=0)
        attr_1 = self.__decode(attr_1)
        attr_2 = self.__decode(attr_2)
        attr_3 = self.__decode(attr_3)
        array = self.__transform(attr_1, attr_2, attr_3, array)
        pred = self.arg_model.predict(array)
        return pred

    def __decode(self, array):
        new_array = []
        label_map = {
            0: "1 (Low)",
            1: "2 (Average)",
            2: "3 (High)",
        }
        for ele in array:
            new_array.append(label_map[np.argmax(ele)])
        return np.array(new_array)

    def __transform(self, attr_1, attr_2, attr_3, array):
        attr_1 = self.__encode(attr_1)
        attr_2 = self.__encode(attr_2)
        attr_3 = self.__encode(attr_3)
        array_new = []
        for idx, ele in enumerate(array):
            temp = np.concatenate((attr_1[idx], attr_2[idx], attr_3[idx], ele))
            array_new.append(temp)
        array = np.array(array_new)
        return array

    def __encode(self, array):
        new_array = []
        label_map = {
            "1 (Low)": np.array([0, 0, 1]),
            "2 (Average)": np.array([0, 1, 0]),
            "3 (High)": np.array([1, 0, 0]),
        }
        for ele in array:
            new_array.append(label_map[ele])
        return np.array(new_array)

### **EVALUATE 2-LAYER MODEL**

In [49]:
# Vectorize the Arguments
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X = X.toarray()

# Extract Overall Quality
y = cleaned_df["overall quality"].to_numpy()
y = y.reshape(-1, )

# Label Encoding
encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)

# Train Test Split
X_train, X_test, y_train, y_test= train_test_split(X, enc_y, test_size=0.2, random_state=110)

# Load Model
custom_model = CustomModel()

# Evaluate Model
pred_train = custom_model.predict(X_train)
pred_test = custom_model.predict(X_test)
print(f"Training Accuracy of Model = {classification_report(y_train, pred_train, output_dict=True)['accuracy']}")
print(f"Validation Accuracy of Model = {classification_report(y_test, pred_test, output_dict=True)['accuracy']}")

Training Accuracy of Model = 0.9759615384615384
Validation Accuracy of Model = 0.6037735849056604


/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr

In [52]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.58      0.85      0.69        26
           1       0.67      0.40      0.50        25
           2       0.00      0.00      0.00         2

    accuracy                           0.60        53
   macro avg       0.42      0.42      0.40        53
weighted avg       0.60      0.60      0.57        53



/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sri/Documents/UTS Research/explainable-ai-in-learning-analytics/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr